<a href="https://colab.research.google.com/github/hydemi/curso-data-science/blob/main/ValidaRDM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U openai langchain-community pypdf langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.8/785.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.98.0
    Uninstalling openai-1.98.0:
      Successfully uninstalled openai-1.98.0


In [2]:
from google.colab import userdata
open_ai_key = userdata.get('open_ai_key')

In [3]:
from openai import OpenAI
client = OpenAI(api_key=open_ai_key)

In [4]:
# Importing necessary modules
import os
#from langchain.document_loaders import PyPDFLoader
#from langchain.prompts import PromptTemplate
#from langchain.vectorstores import FAISS
#from langchain.embeddings.openai import OpenAIEmbeddings
#from langchain.chains import RetrievalQA
#from langchain.llms import OpenAI

Loading the document to the notebook

In [ ]:
# Load the file from https://www.bmj.com/content/331/7531/1498.full.pdf to this enviroment
#!wget https://www.bmj.com/content/331/7531/1498.full.pdf

--2024-07-22 03:11:17--  https://www.bmj.com/content/331/7531/1498.full.pdf
Resolving www.bmj.com (www.bmj.com)... 104.18.32.115, 172.64.155.141
Connecting to www.bmj.com (www.bmj.com)|104.18.32.115|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.bmj.com/content/bmj/331/7531/1498.full.pdf [following]
--2024-07-22 03:11:17--  https://www.bmj.com/content/bmj/331/7531/1498.full.pdf
Reusing existing connection to www.bmj.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘1498.full.pdf.1’

1498.full.pdf.1         [ <=>                ]  92.85K  --.-KB/s    in 0.01s   

2024-07-22 03:11:17 (6.56 MB/s) - ‘1498.full.pdf.1’ saved [95077]



In [11]:
def ask_question(message):
  text = """
1. Critérios de conformidade
Considere “em conformidade” quando o texto:

• Explica claramente o objetivo e o resultado da mudança.
• Identifica a funcionalidade, rotina, tela, erro, serviço ou ativo afetado.
• NÃO descreve passo‑a‑passo técnico (comandos detalhados, scripts, URLs, nomes completos de pacotes).
• PODE citar:
  – nomes de jobs (exemplos CTUJDE00, ECCJDE00) , steps, programas (exemplos CFBSGC, DNCS01, DCBP04) ou elementos de alta plataforma;
  – arquivos, servidores, parâmetros ou **horários internos de lógica** (ex.: “aguardar até 09:30”).
  Esses itens **contextualizam** a mudança e **não** configuram passo‑a‑passo.
• NÃO deve apresentar a **janela de implantação** (data/hora em que a mudança será executada em produção), **a menos que** exista restrição operacional indispensável (“executar exclusivamente entre 01:00 e 02:00”).
• Caso o usuário acrescente uma observação explicando que o horário citado faz parte da lógica ou correção, considere‑o válido.
• Caso o usuário acrescente uma observação explicando que o nome do job ou elemento citado faz parte da lógica ou correção, considere‑o válido.
Observação prática
➜ Referenciar “ALTERAR job DCBJD001 STEP 2” ou “aguardar até 09:30” é permitido, pois indica **o alvo e o parâmetro** da modificação, sem revelar o procedimento técnico detalhado a ser executado.

2. Formato obrigatório da resposta
-Se em conformidade
"Sua descrição atende aos requisitos do checklist"

-Se fora de conformidade
Sua descrição NÃO atende aos requisitos do checklist
Motivo: <<listar exatamente quais regras do checklist foram violadas>>
Sugestões:
<<Exemplo 1 corrigindo a falha>>
<<Exemplo 2 corrigindo a falha>>
Cada exemplo deve corrigir de fato a falha apontada, não apenas parafrasear o texto original.

3. Regras adicionais
Avalie somente o campo informado pelo usuário.
Não inclua cumprimentos, rodapés ou texto fora do formato acima.
Não altere nem replique o conteúdo do “CHECKLIST DE VALIDAÇÃO”; use‑o apenas como referência interna.
Caso o usuário inclua uma observação para esclarecer contexto (ex.: “o horário 09:30 faz parte da lógica do job”), aceite essa observação e trate‑a como informação válida para a análise.

CHECKLIST DE VALIDAÇÃO:
Campo Título >>> Deve informar sucintamente o objetivo da mudança.
Não deve constar procedimentos, pacotes e/ou qualquer outra informação que possua campo específico.
Não incluir especificades de procedimentos.
Não incluir data da execução.
Não utilizar termos como URGENTE para indicar a necessidade de atendimento imediato.
Não deve conter textos como "Atender incidente IN99999" ou "Executar a pedido do gestor".
Quando o campo "Título" contiver informações sucintas que favoreçam o entendimento do que se trata a RDM, ele deverá ser considerada como "Em conformidade".
Exemplos:
Usar: "Atualizar a subrotina de efetivação de PN" ao invés de "Transferir programa CFXSBB"
Usar: "Ampliação do ambiente de servidores do MBK" ao invés de "Adicionar servidores ao pool"
Usar: "Correção da apresentação de extrato PIX no MBK" ao invés de "Correção do erro da IN23311"

Campo Descrição >>> Deve descrever os objetivos e resultados da mudança, com informações da funcionalidade e/ou rotina e/ou tela e/ou erro e/ou serviço e/ou ativo que está sendo tratado.
Não deve constar procedimentos muito detalhados como:
--"Executar o job Jenkins-Deploy-Producao_Mobile para implantar a nova versão"
-- "Copiar o arquivo disponível na URL https://gitlab.brb.com.br/projeto/nome_do_arquivo_completo"
Quando o campo "Descrição" contiver informações detalhadas do que será realizado, o objetivo que se pretente, entre outras informações que favoreçam a análise da RDM ele deverá ser considerada como "Em conformidade".
Informar ainda os serviços que estão sendo afetadas com a mudança.
Exemplos:
Usar: "Correção de subrotina do CFB que estava impedindo a efetivação de PN" ao invés de "Atualizar a subrotina CFBSGC para Produção"
Usar: "Transferência de elementos visando corrigir a inconsistência na rotina de devolução de cheques pelo motivo 49."; ao invés de "Transferir elemento para produção em atendimento à sustentação 20189999999"
Usar: "Correção do JOB CTUJDE00 para inclusão do arquivo AOXACL. A alocação dinâmica do arquivo não está funcionando, estamos alterando o JOB para declarar o arquivo." ao invés de "Transferir JOB CTUJDE00"
Usar: "Atualizar versão do cliente HSM dos servidores SWI3402XX e SWI3402YY. SWI3402XX - Serviços SFC, XYZ / SWI3402YY - Serviços DDA, NPC, ETC"

Para RDMs em ambiente piloto, além do objetivo da implantação, a descrição deve conter as seguintes informações:
- Usuários, grupos de usuários ou unidades que participarão;
- Funcionalidades que serão testadas e avaliadas;
- Servidores ou rotas que serão utilizados;
- Por quanto tempo será testado: xx/xx/201x a xx/xx/201x;
- Horário de funcionamento do piloto: xx:xx às xx:xx;
- Quem irá testar o piloto (área técnica e negocial) - responsável pela validação do sucesso da execução da estratégia ao final do período;
- Qual a funcionalidade/ O que será será testado no piloto;
- Como será desativado (se aplicável).
  """
  # Define a template for the prompt
  prompt_template = f"""
                - Você é um Analista de Mudanças (certificado ITIL Master e COBIT 5 Assessment, QI 150).
                - Sua tarefa é validar solicitações de mudança (RDM) usando o Checklist de Validação e seguindos as instruções, que serão fornecidas a seguir.
                - O usuário poderá solicitar a validação do campo TÍTULO ou DESCRIÇÃO, ou validação de ambos os campos.
                - O resultado da avaliação deverá ser individual. Portanto sua resposta deverá conter o resultado para o TÍTULO e DESCRIÇÃO separadamente.
                - Avalie somente o campo informado pelo usuário.
                - Não inclua cumprimentos, rodapés ou texto fora do formato acima.
                - Não altere nem replique o conteúdo do “CHECKLIST DE VALIDAÇÃO”; use‑o apenas como referência interna.
                - Caso o usuário inclua uma observação para esclarecer contexto (ex.: “o horário 09:30 faz parte da lógica do job”), aceite essa observação e trate‑a como informação válida para a análise.

                ###INSTRUÇÕES E CHECKLIST DE VALIDAÇÃO###
                ```{text}```
                """
  #prompt = PromptTemplate.from_template(prompt_template)
  response = client.chat.completions.create(
      messages=[

          {
              "role": "system",
              "content": prompt_template
          },
          {
              "role": "user",
              "content": f"Valide o conteúdo informado de acordo com as instruções: {message}"
          },
      ],
      model="gpt-3.5-turbo",
      temperature=0.7
  )
  return response

In [12]:
response = ask_question("""Campo: 'Descrição' / Conteúdo: Com intuito de manter a operação dos clusters de Firewall em alta performance, esta mudança propõe a busca de regras Zero Hit para higienização do ambiente e manutenção da base de regras de filtragem otimizada, proporcionando maior eficiência e performance na compilação, aplicação de regras e analise de pacotes nos clusters de firewall do Banco.Escopo da execução:Para desabilitar regras Zero Hits:SFW340210-11SFW340214-15Existe necessidade de parar monitoramento? Não.Existe necessidade de envio de comunicado? Não.Existe necessidade de ajustar o mapeamento do NUCOF? Não.Há risco de impacto? Se tiver risco, qual impacto em caso de incidente? Não, porque serão desativadas apenas as regras sem hit dentro de um período de observação mínimo de 6 meses.Quando houver execução de atividade por outra área técnica: A atividade foi devidamente alinhada com a área técnica envolvida?""")
print(response.choices[0].message.content)

Campo Descrição:
"Sua descrição atende aos requisitos do checklist"


In [17]:
import pandas as pd

# Consultar o arquivo RDMsAgosto.xlsx em "sample_data"
df = pd.read_excel("/content/sample_data/RDMsAgosto.xlsx")

# Exibir as primeiras linhas do DataFrame
df.head()

,NUMBER,BRIEF_DESCRIPTION,DESCRIPTION
0,RDM90282,Sistema de gestão de tarifas,Criação de tabelas para atender a necessidade ...
1,RDM90283,"SAP ECC - Cadastramento PA's 393, 398, 399","U823161 TL - CRIAÇÃO DE CENTRO, LOCAL DE ..."
2,RDM90285,Apontamento HRX-ABX Linux,Vamos voltar o apontamento dos trancodes 260 e...
3,RDM90293,Execução de script DML,Execução de DML sem interrupção de serviço com...
4,RDM90284,Ajuste para atendimento de solicitações do ECC...,Correção do programa ECCPJW e para resolver pr...


In [ ]:
#Para cada registro em df, solicitar avaliação da IA.
#BRIEF_DESCRIPTION = Titulo
#DESCRIPTION = Descrição

for rdm, descricao, titulo in zip(df['NUMBER'], df['BRIEF_DESCRIPTION'], df['DESCRIPTION']):
  response = ask_question(f"""Campo: 'Descrição' / Conteúdo: {descricao}
                          Campo: 'Título' / Conteúdo: {titulo}""")

response = ask_question("""Campo: 'Descrição' / Conteúdo: Com intuito de manter a operação dos clusters de Firewall em alta performance, esta mudança propõe a busca de regras Zero Hit para higienização do ambiente e manutenção da base de regras de filtragem otimizada, proporcionando maior eficiência e performance na compilação, aplicação de regras e analise de pacotes nos clusters de firewall do Banco.Escopo da execução:Para desabilitar regras Zero Hits:SFW340210-11SFW340214-15Existe necessidade de parar monitoramento? Não.Existe necessidade de envio de comunicado? Não.Existe necessidade de ajustar o mapeamento do NUCOF? Não.Há risco de impacto? Se tiver risco, qual impacto em caso de incidente? Não, porque serão desativadas apenas as regras sem hit dentro de um período de observação mínimo de 6 meses.Quando houver execução de atividade por outra área técnica: A atividade foi devidamente alinhada com a área técnica envolvida?""")
print(response.choices[0].message.content)